In [1]:
import cv2
from skimage.feature import hog
import numpy as np
import mahotas
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skopt import gp_minimize
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
labels = np.load('labels_SVM.npy')
features = np.load('features.npy')

In [3]:
def color_hist(input_image, nbins=32):
    ch1 = np.histogram(input_image[:,:,0], bins = nbins, range = (0, 256))[0] # [0] is because we need only the histogram, not bins edges
    ch2 = np.histogram(input_image[:,:,1], bins = nbins, range = (0, 256))[0]
    ch3 = np.histogram(input_image[:,:,2], bins = nbins, range = (0, 256))[0]
    return np.hstack((ch1, ch2, ch3))

In [4]:
def extract_features(input_image, nbins, orient, pix_per_cell, cell_per_block):
    output = []
    for img in input_image:
        img_ = np.asarray(Image.fromarray(img).convert('L'))
        features = hog(img_, orientations = orient, pixels_per_cell = (pix_per_cell, pix_per_cell), 
                       cells_per_block = (cell_per_block, cell_per_block), 
                       multichannel=False, transform_sqrt = True, visualize = False, feature_vector = True )
        output.append(np.hstack((cv2.resize(img, dsize=(16,16)).ravel(),color_hist(img, nbins),features)))
    return np.array(output)


In [5]:
nbins, orient, pix_per_cell, cell_per_block = [64, 8, 4, 1]
features_ = extract_features(features, nbins, orient, pix_per_cell, cell_per_block)
y = labels
X = features_                    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=22)

In [6]:
svc = LinearSVC(dual=True, C=1, max_iter=10)
svc.fit(X_train, y_train)
test_acc = svc.score(X_test, y_test)
print(test_acc)

0.9605967865340475


/home/darius/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [7]:
svc.score(X_train, y_train)

0.9833583065004623

In [ ]:
img_ = np.asarray(Image.fromarray(features[2]).convert('L'))
features_ = hog(img_, orientations = orient, pixels_per_cell = (pix_per_cell, pix_per_cell), 
               cells_per_block = (cell_per_block, cell_per_block), 
               multichannel=False, transform_sqrt = True, visualize = False, feature_vector = True )
print(features_.shape)

In [ ]:
svc = SVC(gamma = 'auto', cache_size = 2048)
svc.fit(X_train, y_train)
test_acc = svc.score(X_test, y_test)

In [ ]:
space  = [(8, 32),                  # Number of bins for color histogram
          (4, 8),                  # HOG number of orientations
          (4, 8),                  # HOG pixels per cell
          (1, 2)]                   # HOG cells per block
i = 0

def obj(params):
    global i
    nbins, orient, pix_per_cell, cell_per_block = params
    # Use only every 10th images to speed things up.
    features_ = extract_features(features[::20,:,:,:], nbins, orient, pix_per_cell, cell_per_block)
    y = labels[::20]
    X = features_                    
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=22)
    svc = LinearSVC()
    svc.fit(X_train, y_train)
    test_acc = svc.score(X_test, y_test)
    print (i, params, test_acc)
    i += 1
    return 1.0 - test_acc
    
res = gp_minimize(obj, space, n_calls = 20, random_state = 22)
print("Best score=%.4f" % res.fun)